# Test building load profile scenario reduction

In [1]:
import os
import numpy as np
import pandas as pd
import itertools
from scenarioReducer import Fast_forward

In [2]:
# set up params
years = list(range(2012, 2018))
ids = [0, 4, 8, 19, 25, 40, 58, 102, 104, 118]

data_dir = os.path.join('data','processed')
building_file_pattern = 'ly_{id}-{year}.csv'

n_buildings = 8

In [3]:
# sample vectors of building-years (scnearios)
np.random.seed(0)
n_samples = 10000
scenarios = np.array([list(zip(np.random.choice(ids, n_buildings),np.random.choice(years, n_buildings))) for _ in range(n_samples)])

In [4]:
# load building-year load profiles once to reduce I/O time
load_profiles = {
    f'{building_id}-{year}': pd.read_csv(
        os.path.join(data_dir, building_file_pattern.format(id=building_id, year=year)),
        usecols=['Equipment Electric Power [kWh]']
        )['Equipment Electric Power [kWh]'].to_numpy()\
            for building_id, year in itertools.product(ids, years)
}

In [5]:
def get_scenario_stats(building_year_vector, load_profiles):
    """Compute standard deviation and peak of aggregate load for a given scenario,
    i.e. set of building-year profiles (profiles for each building in scenario)."""

    aggregate_load = np.sum([load_profiles[f'{building_id}-{year}'] for building_id, year in building_year_vector], axis=0)

    return np.std(aggregate_load), np.max(aggregate_load)

In [6]:
def rescale_array(a, invert=False, bounds=None):
    assert not (invert and bounds is None), 'bounds must be provided when inverting scaling.'
    old_bounds = (a.min(), a.max()) if not invert else (0, 1)
    new_bounds = (0, 1) if not invert else bounds
    return np.interp(a, old_bounds, new_bounds)

In [7]:
scenario_stats = np.array([get_scenario_stats(scenario, load_profiles) for scenario in scenarios])

In [8]:
print(scenario_stats)

[[ 215.49388094 1609.7       ]
 [ 279.10132837 1986.9       ]
 [ 289.17318987 1975.5       ]
 ...
 [ 335.80176959 1933.        ]
 [ 273.04669928 1738.9       ]
 [ 182.27858895 1530.8       ]]


In [9]:
bounds = [(np.min(scenario_stats[:,0]), np.max(scenario_stats[:,0])),(np.min(scenario_stats[:,1]), np.max(scenario_stats[:,1]))]
print(bounds)

[(91.85282087503722, 461.89710322384803), (1079.7, 2779.7000000000003)]


In [10]:
# rescale scenario stats to range [-1,1] for each exis
scaled_scenario_stats = np.array([rescale_array(scenario_stats[:,0]), rescale_array(scenario_stats[:,1])]).T

## Perform scenario reduction

In [11]:
probs = np.ones(shape=n_samples)/n_samples # uniform probability of scenarios
num_reduced_scenarios = 10

FFreducer = Fast_forward(scaled_scenario_stats.T, probs)

In [12]:
reduced_scenario_stats, reduced_probs, reduced_indices = FFreducer.reduce(distance=1,n_scenarios=num_reduced_scenarios)
# use 1-norm distance metric for reduction
reduced_scenario_stats = reduced_scenario_stats.T
reduced_scenarios = scenarios[reduced_indices]

In [13]:
rescaled_reduced_scenario_stats = np.array([rescale_array(reduced_scenario_stats[:,0], invert=True, bounds=bounds[0]), rescale_array(reduced_scenario_stats[:,1], invert=True, bounds=bounds[1])]).T

In [14]:
# check reduced scenarios have been accessed correctly
assert np.isclose(rescaled_reduced_scenario_stats, scenario_stats[reduced_indices]).all()

In [15]:
print(rescaled_reduced_scenario_stats)

[[ 257.62163361 1734.2       ]
 [ 316.5369086  1938.4       ]
 [ 200.85413096 1533.4       ]
 [ 361.65929966 2072.6       ]
 [ 230.090032   1641.2       ]
 [ 289.55366208 1794.4       ]
 [ 163.95969852 1393.8       ]
 [ 264.74788101 1906.1       ]
 [ 222.9136756  1768.5       ]
 [ 296.12537073 2118.1       ]]


In [16]:
print(scenarios[reduced_indices])

[[[  25 2013]
  [  19 2013]
  [ 118 2015]
  [ 102 2012]
  [  19 2015]
  [   0 2012]
  [  40 2012]
  [   0 2015]]

 [[ 102 2015]
  [ 118 2016]
  [ 104 2015]
  [   8 2013]
  [ 102 2017]
  [   8 2015]
  [   4 2013]
  [   4 2015]]

 [[  40 2014]
  [ 118 2015]
  [ 104 2015]
  [  58 2012]
  [  25 2016]
  [  25 2016]
  [   8 2014]
  [  58 2014]]

 [[ 102 2017]
  [   0 2012]
  [  19 2012]
  [   0 2015]
  [ 102 2015]
  [   8 2013]
  [   0 2014]
  [   0 2017]]

 [[  25 2012]
  [  19 2012]
  [   4 2017]
  [  58 2014]
  [ 104 2014]
  [  25 2016]
  [   0 2012]
  [  19 2012]]

 [[   0 2017]
  [   4 2016]
  [  19 2017]
  [  58 2017]
  [   4 2014]
  [  58 2013]
  [   0 2013]
  [ 118 2014]]

 [[  40 2013]
  [  40 2014]
  [  40 2017]
  [  25 2014]
  [  58 2017]
  [   4 2017]
  [  25 2015]
  [  58 2017]]

 [[ 118 2014]
  [  19 2016]
  [  25 2015]
  [ 104 2014]
  [   0 2014]
  [ 102 2014]
  [ 118 2016]
  [ 104 2016]]

 [[  25 2012]
  [  25 2013]
  [  40 2017]
  [ 118 2012]
  [  25 2017]
  [  19 2017]
  [ 